In [0]:
from pyspark.sql import SparkSession
import os

# Iniciando uma SparkSession com Delta Lake
spark = SparkSession.builder \
    .appName("GoldLayer") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Adicionando configurações S3 diretamente na sessão Spark
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY"))
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", os.getenv("AWS_SECRET_KEY"))
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

# Configurações do bucket S3
gold_bucket = "s3a://engenharia-dados-satc-gold-bucket"

# Carregar arquivos Delta da camada gold
datas = spark.read.format("delta").load(f'{gold_bucket}/dim_data')
entregas = spark.read.format("delta").load(f'{gold_bucket}/dim_entregas')
pedidos = spark.read.format("delta").load(f'{gold_bucket}/fato_pedidos')
transacoes = spark.read.format("delta").load(f'{gold_bucket}/dim_transacao')
localizacoes = spark.read.format("delta").load(f'{gold_bucket}/dim_localizacoes')

In [0]:
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME =os.getenv("DB_NAME")

jdbc_url = f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}"
connection_properties = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "driver": "org.postgresql.Driver"
}

datas.write.jdbc(url=jdbc_url, table="dw.dim_data", mode="overwrite", properties=connection_properties)
entregas.write.jdbc(url=jdbc_url, table="dw.dim_entregas", mode="overwrite", properties=connection_properties)
pedidos.write.jdbc(url=jdbc_url, table="dw.fato_pedidos", mode="overwrite", properties=connection_properties)
transacoes.write.jdbc(url=jdbc_url, table="dw.dim_transacoes", mode="overwrite", properties=connection_properties)
localizacoes.write.jdbc(url=jdbc_url, table="dw.dim_localizacoes", mode="overwrite", properties=connection_properties)
